In [ ]:
%load_ext autoreload
%autoreload 2

from coeditor.common import *
from coeditor.dataset import *
from spot.utils import pretty_print_dict

os.chdir(proj_root())

In [ ]:
dataset_name = "medium"
save_dir = get_dataset_dir(dataset_name) / "tokenized-file_based"
datasets = load_datasets(save_dir)

In [ ]:
for group, dataset in datasets.items():
    dataset = pickle_load(save_dir / f"{group}.pkl")
    print("=" * 20, group, "=" * 20)
    pretty_print_dict(dataset.overall_stats())

In [ ]:
for group in datasets:
    if not (save_dir / f"{group}.pkl").exists():
        continue
    dataset = pickle_load(save_dir / f"{group}.pkl")
    print("=" * 20, group, "=" * 20)
    display(dataset.per_repo_stats())

In [ ]:
print(list(datasets["test"].all_edits())[1].show_prediction())